In [1]:
pip install torch torchvision torchaudio


  Using cached torchvision-0.20.1-cp312-cp312-manylinux1_x86_64.whl.metadata (6.1 kB)
  Using cached torchaudio-2.5.1-cp312-cp312-manylinux1_x86_64.whl.metadata (6.4 kB)
  Using cached pillow-11.0.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (9.1 kB)
   ━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/7.2 MB 24.6 kB/s eta 0:04:02^C
   ━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/7.2 MB 24.6 kB/s eta 0:04:02
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.models import mobilenet_v2

# 定义数据转换
transform = transforms.Compose([
    transforms.Resize((224, 224)),   # 将 28x28 调整为 224x224 适用于 MobileNetV2
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # MNIST 数据集标准化参数
])

# 加载 MNIST 数据集
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)

# 加载 MobileNetV2 模型
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = mobilenet_v2(weights=None)  # 不使用预训练权重
model.classifier[1] = nn.Linear(model.last_channel, 10)  # MNIST 有 10 个类别
model = model.to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练模型
epochs = 5
model.train()
for epoch in range(epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader, 0):
        inputs, labels = inputs.to(device), labels.to(device)

        # 前向传播
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # 反向传播
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:  # 每 100 个 batch 打印一次 loss
            print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}")
            running_loss = 0.0

print('Finished Training')
torch.save(model.state_dict(), 'mobilenet_v2_mnist.pth')  # 保存模型


ModuleNotFoundError: No module named 'torch'

In [ ]:
import torch
from torchvision import transforms
from torchvision.models import mobilenet_v2
from PIL import Image

# 加载 MobileNetV2 模型
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = mobilenet_v2(weights=None)
model.classifier[1] = torch.nn.Linear(model.last_channel, 10)  # MNIST 有 10 个类别
model.load_state_dict(torch.load('mobilenet_v2_mnist.pth'))  # 加载之前保存的模型
model = model.to(device)
model.eval()

# 定义数据转换
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# 加载测试图像
image = Image.open('test_digit.png').convert('L')  # 请使用您本地的测试手写数字图像
input_tensor = transform(image).unsqueeze(0).to(device)

# 推理
with torch.no_grad():
    output = model(input_tensor)
    _, predicted = torch.max(output, 1)
    print(f'Predicted: {predicted.item()}')  # 输出预测类别


In [ ]:
nsys profile -o train_mnist_output python train_mnist.py


In [ ]:
nsys profile -o inference_mnist_output python inference_mnist.py
